In [1]:
import pandas as pd
import blocksci
import numpy as np
import zarr
import networkx as nx

DIR_PARSED="/mnt/hdd_data/blockchain_parsed/"
cfg_file = f"/mnt/hdd_data/blockchain_parsed/bitcoin.cfg" # later
chain = blocksci.Blockchain(cfg_file)

# scope
this notebook goal is to replicate some results from `paint-it-black` as a sanity check.
the original paper analysed form block 1 to block 400.000

# 1: Number of addresses per darknet
here we want to replicate table 1 from `pait-it-black`: 
measure the number of addresses for each entity

In [24]:
df2 = pd.read_csv(f"/home/nvalla/paint-black/uniform_black/heur_2_data/ground_truth_clust_id.csv")

In [25]:
entities_c = df2['entity'].value_counts()

In [26]:
print(entities_c)

EvolutionMarket           419915
SilkRoadMarketplace       372553
SilkRoad2Market           349836
AlphaBayMarket            196827
NucleusMarket             146281
AbraxasMarket             119065
PandoraOpenMarket          55357
SheepMarketplace           53639
BlackBankMarket            50878
MiddleEarthMarketplace     34049
BlueSkyMarketplace         18897
CannabisRoadMarket          2829
BabylonMarket               1471
GreenRoadMarket              965
Name: entity, dtype: int64


The number of addresses per entity is close enough to table 1 from [].

# 2: Number of input and output transactions per darkent
here we replicate table 2 from `paint-it-black`:
Table showing the number of transactions received
and transactions sent from and to each of the 14 darknets.

In [ ]:
df2['output_txes'] = df2['address'].apply(lambda a: chain.address_from_string(a).output_txes_count())

df2.groupby(['entity'])['output_txes'].sum()

df2['input_txes'] = df2['address'].apply(lambda a: chain.address_from_string(a).input_txes_count())

df2.groupby(['entity'])['input_txes'].sum()

In [ ]:
"""
l = [(chain.address_from_string(a).input_txes_count(), chain.address_from_string(a).output_txes_count()) for a in df.address]

df["input_txes_count"] = [a[0] for a in l]
df["output_txes_count"] = [a[1] for a in l]

df.groupby(['entity'])["input_txes_count"].sum()

df.groupby(['entity'])["output_txes_count"].sum()
"""
#_this method doesnt work: may be counting transactions more than once(if two different address are in the same transaction) per entity. Table 2 caption is not clear enough._

# 3:
table 3 from paint-it-black

In [19]:
df1 = pd.read_csv(f"/home/nvalla/paint-black/uniform_black/heur_1_data/ground_truth_clust_id.csv")

In [20]:
a = df1.groupby(['entity'])["cluster_id"].nunique()
print(a)

entity
AbraxasMarket               1
AlphaBayMarket              2
BabylonMarket               1
BlackBankMarket             1
BlueSkyMarketplace          1
CannabisRoadMarket          1
EvolutionMarket           121
GreenRoadMarket             1
MiddleEarthMarketplace      1
NucleusMarket               1
PandoraOpenMarket           1
SheepMarketplace            1
SilkRoad2Market             1
SilkRoadMarketplace         1
Name: cluster_id, dtype: int64


In [22]:
a = df1.groupby(['entity'])["cluster_id"].apply(lambda x: np.unique(x, return_counts=True))
print(a)

entity
AbraxasMarket                                       ([101080866], [119065])
AlphaBayMarket                     ([107302595, 115726064], [7151, 189676])
BabylonMarket                                          ([94998167], [1471])
BlackBankMarket                                       ([90613521], [50878])
BlueSkyMarketplace                                    ([88316586], [18897])
CannabisRoadMarket                                     ([92044383], [2829])
EvolutionMarket           ([89576173, 93820427, 93821719, 93822603, 9428...
GreenRoadMarket                                        ([112150330], [965])
MiddleEarthMarketplace                                ([94494584], [34049])
NucleusMarket                                        ([99422262], [146281])
PandoraOpenMarket                                     ([86826333], [55357])
SheepMarketplace                                      ([82285475], [53639])
SilkRoad2Market                                      ([87290351], [349836])
SilkR

# 4: tabella 4
table 4 from paint-it-black

In [9]:
df2 = pd.read_csv(f"/home/nvalla/paint-black/uniform_black/heur_2_data/ground_truth_clust_id.csv")

In [10]:
df2.shape[0]

1822562

In [11]:
b = df2.groupby(['entity'])["cluster_id"].nunique()
print(b)

entity
AbraxasMarket               1
AlphaBayMarket              1
BabylonMarket               1
BlackBankMarket             1
BlueSkyMarketplace          1
CannabisRoadMarket          1
EvolutionMarket           119
GreenRoadMarket             1
MiddleEarthMarketplace      1
NucleusMarket               1
PandoraOpenMarket           1
SheepMarketplace            1
SilkRoad2Market             1
SilkRoadMarketplace         1
Name: cluster_id, dtype: int64


In [18]:
b = df2.groupby(['entity'])["cluster_id"].apply(lambda x: np.unique(x, return_counts=True))
print(b)

entity
AbraxasMarket                                               ([5], [119065])
AlphaBayMarket                                              ([5], [196827])
BabylonMarket                                                 ([5], [1471])
BlackBankMarket                                              ([5], [50878])
BlueSkyMarketplace                                           ([5], [18897])
CannabisRoadMarket                                            ([5], [2829])
EvolutionMarket           ([5, 88463297, 88463462, 88464545, 88860818, 8...
GreenRoadMarket                                                ([5], [965])
MiddleEarthMarketplace                                       ([5], [34049])
NucleusMarket                                               ([5], [146281])
PandoraOpenMarket                                            ([5], [55357])
SheepMarketplace                                             ([5], [53639])
SilkRoad2Market                                             ([5], [349836])
SilkR

b relates each entity to two list: first list the cluster id, second list the appearances for each cluster id.
All entities belong to the same user but one.

# number of clusters 
number of clusters black from the ground truth

In [2]:
clust_is_black = zarr.load("uniform_black/heur_2_data/cluster_is_black_ground_truth.zarr")
print(clust_is_black.sum())

119


In [3]:
clust_is_black_when = zarr.load("uniform_black/heur_2_data/cluster_is_black_final_day.zarr")

In [2]:
df = pd.read_csv("uniform_black/heur_2_data/diffusion_net_day.csv")

In [7]:
cm = blocksci.cluster.ClusterManager(DIR_PARSED + "bitcoin/heur_2/", chain)

In [24]:
add = chain.address_from_string("17DEJSAMHsTtsxZA1v6yu51xVwtFJ5ECfp")
clu = cm.cluster_with_address(add)  # cluster index 5

blocks = []
for t in add.input_txes:
    blocks.append(t.block_height)
    
b = chain.blocks[blocks[0]]
print(b.time)
date = '2015-08-26'

In [25]:
g = nx.read_graphml(f'{DIR_PARSED}bitcoin/heur_2_networks_day/{date}.graphml.bz2')

In [27]:
'5' in g.nodes

True

In [30]:
df.loc[df.date == date]

,date,no_old_black_user,no_new_black_user,no_active_black_user,no_cum_black_user,black2new_no_trx,black2new_value,black2new_no_link,black2black_no_trx,black2black_value,black2black_no_link,not_black_link
2426,2015-08-26,0,0,0,0,0,0,0,0,0,0,170794


# simulate ub_diffusion_net on one day network

In [ ]:
g = nx.read_graphml(f'{DIR_PARSED}bitcoin/heur_2_networks_day/{date}.graphml.bz2')

In [54]:
clust_is_black_ground = zarr.load(f"uniform_black/heur_2_data/cluster_is_black_ground_truth.zarr")
clust_is_black_ground_set = set([str(i) for i in range(len(clust_is_black_ground)) if clust_is_black_ground[i]])

In [75]:
new_black_nodes = set([])
old_black_nodes = clust_is_black_ground_set.intersection(g.nodes)
print(old_black_nodes)
clust_is_black_when = np.zeros(len(clust_is_black_ground), dtype=str)

black2new_no_trx = 0
black2new_value = 0
black2new_link = 0
black2black_no_trx = 0
black2black_value = 0
black2black_link = 0

s = 0
for black_node in old_black_nodes:
    # check out-neighbours of black node
    for e in g[black_node]:
        # if not already black, track it
        if e not in old_black_nodes:
            new_black_nodes.update([e])
            black2new_no_trx += g[black_node][e]['n_tx']
            black2new_value += g[black_node][e]['value']
            black2new_link += 1
        else:
            black2black_no_trx += g[black_node][e]['n_tx']
            black2black_value += g[black_node][e]['value']
            black2black_link += 1
        if clust_is_black_when[int(e)] == '':
            clust_is_black_when[int(e)] = date
            s +=1


{'5'}


In [76]:
print(s)
np.count_nonzero(clust_is_black_when)

26028


26028